In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd


#load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

#save the file
with open('hamlet.txt','w') as file:
  file.write(data)



[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/venkatesha/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [2]:
#Data preprocessing
import tensorflow
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#load the dataset
with open('hamlet.txt','r') as file:
  text=file.read().lower()

#Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [ ]:
tokenizer.word_index

In [ ]:
#Create I/P sequence
input_sequence = []
for line in text.split('\n'):
  tokenize_lists = tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(tokenize_lists)):
    n_gram_list = tokenize_lists[:i+1]
    input_sequence.append(n_gram_list)
input_sequence

In [5]:
# pad sequence
max_len = max(len(x) for x in input_sequence)
max_len

14

In [6]:
input_sequence = np.array(pad_sequences(input_sequence, maxlen = max_len))
input_sequence

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [7]:
#Create predictore and label
import tensorflow as tf
x,y = input_sequence[:,:-1], input_sequence[:,-1]
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [8]:
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [9]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
#split train and test data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [ ]:
#We can also impement in GRU
#Train LSTM model
'''from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GRU

model = Sequential()
model.add(Embedding(total_words,100))
model.add(GRU(150,return_sequences = True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words,activation='softmax'))

#compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()'''

In [11]:
#Train LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM

model = Sequential()
model.add(Embedding(total_words,100))
model.add(LSTM(150,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

#compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
#Train Model
history = model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.0292 - loss: 7.1164 - val_accuracy: 0.0356 - val_loss: 6.7774
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0354 - loss: 6.4527 - val_accuracy: 0.0414 - val_loss: 6.8398
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0441 - loss: 6.2839 - val_accuracy: 0.0460 - val_loss: 6.8777
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0544 - loss: 6.1424 - val_accuracy: 0.0505 - val_loss: 6.8998
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0555 - loss: 6.0276 - val_accuracy: 0.0517 - val_loss: 6.9319
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0615 - loss: 5.8844 - val_accuracy: 0.0561 - val_loss: 6.9766
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0682 - loss: 5.7469 - val_accuracy: 0.0633 - val_loss: 7.0431
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0799 - loss: 5

In [21]:
#Function of next Prediction word
def predict_next_word(model, tokenizer, text, max_len):
    token_lists = tokenizer.texts_to_sequences([text])[0]
    if len(token_lists) >= max_len:
        token_lists = token_lists[-(max_len-1):]
    token_lists = pad_sequences([token_lists], maxlen=max_len-1,padding='pre')
    predicted = model.predict(token_lists,verbose=1)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [27]:
input_text = 'Tis now strook twelue, get thee to bed'
print(input_text)
max_len = model.input_shape[1]+1
next_word = predict_next_word(model, tokenizer, input_text, max_len)
print(f'Next word is:{next_word}')

Tis now strook twelue, get thee to bed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Next word is:francisco


In [28]:
#Save the model
model.save('predict_next_word_lstm.h5')

#Save the tokenizer
import pickle

with open('tokenizer.pickel','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)